In [1]:
from Pegasus.api import *

In [2]:
# not required. Just to view output from certain tools.
from pathlib import Path
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
nums1 = File("nums1.txt")
nums2 = File("nums2.txt")
nums3 = File("nums3.txt")
nums4 = File("nums4.txt")

rc = ReplicaCatalog()\
    .add_replica("local", nums1, Path(".").resolve() / "nums1.txt")\
    .add_replica("local", nums2, Path(".").resolve() / "nums2.txt")\
    .add_replica("local", nums3, Path(".").resolve() / "nums3.txt")\
    .add_replica("local", nums4, Path(".").resolve() / "nums4.txt")\
    .write()

In [4]:
tc =  TransformationCatalog()

separate = Transformation(
            "separate",
            site="local",
            pfn="/home/scitech/shared-data/separate.py",
            is_stageable=True,
            arch=Arch.X86_64,
            os_type=OS.LINUX
)

count = Transformation(
            "count",
            site="local",
            pfn="/home/scitech/shared-data/count.py",
            is_stageable=True,
            arch=Arch.X86_64,
            os_type=OS.LINUX
)

tar = Transformation(
            "tar",
            site="condorpool",
            pfn="/bin/tar",
            is_stageable=False,
            arch=Arch.X86_64,
            os_type=OS.LINUX
)

tc.add_transformations(separate, count, tar)
tc.write()

In [5]:
wf = Workflow("diamomnd")

odd_nums = File("odd_nums.txt")
even_nums = File("even_nums.txt")

job_separate = Job(separate)\
                .add_inputs(nums1, nums2, nums3, nums4)\
                .add_outputs(odd_nums, even_nums)

odd_count = File("odd_count.txt")
job_count_odd = Job(count)\
                .add_args(odd_nums, odd_count)\
                .add_inputs(odd_nums)\
                .add_outputs(odd_count)

even_count = File("even_count.txt")
job_count_even = Job(count)\
                .add_args(even_nums, even_count)\
                .add_inputs(even_nums)\
                .add_outputs(even_count)

tar_result = File("result.tar.gz")
job_tar = Job(tar)\
          .add_args("-cvf", tar_result, even_count, odd_count)\
          .add_inputs(even_count, odd_count)\
          .add_outputs(tar_result)
            
wf.add_jobs(job_separate, job_count_odd, job_count_even, job_tar)

In [6]:
try:
    wf.plan(submit=True)\
        .wait()
except PegasusClientError as e:
    print(e)


################
# pegasus-plan #
################
2021.02.03 02:35:06.244 UTC:
2021.02.03 02:35:06.250 UTC:   -----------------------------------------------------------------------
2021.02.03 02:35:06.255 UTC:   File for submitting this DAG to HTCondor           : diamomnd-0.dag.condor.sub
2021.02.03 02:35:06.260 UTC:   Log of DAGMan debugging messages                 : diamomnd-0.dag.dagman.out
2021.02.03 02:35:06.266 UTC:   Log of HTCondor library output                     : diamomnd-0.dag.lib.out
2021.02.03 02:35:06.271 UTC:   Log of HTCondor library error messages             : diamomnd-0.dag.lib.err
2021.02.03 02:35:06.277 UTC:   Log of the life of condor_dagman itself          : diamomnd-0.dag.dagman.log
2021.02.03 02:35:06.282 UTC:
2021.02.03 02:35:06.287 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2021.02.03 02:35:06.298 UTC:   -----------------------------------------------------------------------
2021.02.03 02:35:08.754 UTC:   Your data

[####################################] 100.0% ..Success (Completed: 19, Queued: 0, Running: 0, Failed: 0)
